In [1]:
import pandas as pd
import numpy as np
import import_ipynb
import dividend_detective_scrape

import yfinance as yf

import datetime as dt

from datetime import datetime

importing Jupyter notebook from dividend_detective_scrape.ipynb


In [2]:
%%time
div_stocks = dividend_detective_scrape.get_data()

CPU times: user 28.1 ms, sys: 15.6 ms, total: 43.7 ms
Wall time: 14.2 s


In [3]:
div_stocks

,ticker,name,annual_div,div_yield
0,FCCY,1st Constitution Bancorp,0.36,2.8
1,SRCE,1st Source,1.12,3.4
2,MMM,3M Company,5.88,4.0
3,ABBV,AbbVie,4.72,5.6
4,ACCO,ACCO Brands,0.26,4.1
...,...,...,...,...
740,WYND,Wyndham Destinations,1.85,7.3
741,WH,Wyndham Hotels & Resorts,1.28,2.9
742,XEL,Xcel Energy,1.72,2.9
743,XRX,Xerox Holdings,1.00,5.5


In [4]:
min_div = 5.0

approved_div_stocks = div_stocks[div_stocks['div_yield']>min_div]

approved_div_stocks

,ticker,name,annual_div,div_yield
3,ABBV,AbbVie,4.72,5.6
6,ADES,Advanced Emissions Solutions,1.00,13.4
9,AGNC,AGNC Investment,1.44,11.3
11,ALX,Alexander's,18.00,7.1
13,AB,Alliancebernstein Holding,2.68,11.8
...,...,...,...,...
735,WMB,Williams Companies,1.60,8.2
739,WPC,WP Carey,4.16,6.7
740,WYND,Wyndham Destinations,1.85,7.3
743,XRX,Xerox Holdings,1.00,5.5


In [5]:
div_stocks['div_consistency_score'] = 0.0

In [6]:
def trim_div_data(stockObj):
    actns = stockObj.actions
    for i,r in actns.iterrows():
        if r['Dividends'] > 0:
            first_div_date = i
            break

    actns = actns.loc[first_div_date:]
    return actns, first_div_date

In [7]:
def add_div_data_details(actns, first_div_date):
    actns['days_since_first_div'] = (actns.index-first_div_date).days
    actns['year'] = actns.index.year
    
    h = []
    l = []
    for i,r in actns.iterrows():
        prices = stockObj.history(start=i,end = i+pd.DateOffset(1)).iloc[0]
        h.append(prices['High'])
        l.append(prices['Low'])

    actns['Stock_High'] = h
    actns['Stock_Low'] = l
    
    actns['day_div_yield'] = actns['Dividends']/actns['Stock_High']
    return actns

In [8]:
def get_ind_div_data(stockObj):
    df, first_div_date = trim_div_data(stockObj)
    return add_div_data_details(df, first_div_date)

In [9]:
def get_div_consistency_score(stockObj):
    
    df = stockObj.actions
    df['year'] = df.index.year
    df = df[df['Dividends']>0]
    df = df.drop(columns=['Stock Splits'])
    
    ann_df = df.groupby(['year']).agg(
        {'Dividends': np.sum}).rename(columns=
        {'Dividends':'dividends'})
    start_year = ann_df.index[0]
    end_year = ann_df.index[len(ann_df)-1]+1
    ann_df = ann_df.reindex(list(range(start_year,end_year)), fill_value=0)
    
    consistancy_coeff = [1/(2**(x+1)) for x in range(len(ann_df))][::-1]
    year_progress = datetime.now().timetuple().tm_yday/365
    consistancy_coeff[-1] = consistancy_coeff[-1]*year_progress
    
    ann_df['div_consistency_score'] = consistancy_coeff
    
    highest_div = ann_df.iloc[0]['dividends']
    
#     add another weight to the last cell where i get the percentage of the year we've been in and multiply it by that

    for i,r in ann_df[1:].iterrows():      
#         i need to think about a better way to do this because right now
#         it compares the wrong two pieces
        if r['dividends'] < ann_df.loc[i-1]['dividends']:
            mult = r['div_consistency_score']
            div_perc = r['dividends']/highest_div
            ann_df.at[i, 'div_consistency_score'] = mult * div_perc
        else:
            highest_div = r['dividends']
    
    return ann_df['div_consistency_score'].sum()

In [10]:
def get_all_consistancy_scores(stockList):
    for index, row in stockList.iterrows():
        stockObj = yf.Ticker(row['ticker'])
        div_consis = get_div_consistency_score(stockObj)
        stockList.at[index, 'div_consistency_score'] = div_consis
    return stockList

In [11]:
%%time
acceptable_div_stocks = get_all_consistancy_scores(approved_div_stocks)

/Users/jackselbo/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/jackselbo/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


KeyboardInterrupt: 